In [1]:
import sys
print(sys.version) # Print Python version -- On Vocareum, it should be 3.7+

from IPython.display import display
import pandas as pd

from nb7utils import canonicalize_tibble, tibbles_are_equivalent, cast

3.8.7 (default, Jan 25 2021, 11:14:52) 
[GCC 5.5.0 20171010]


In [2]:
from nb9utils import download, get_path, auxfiles

for filename, checksum in auxfiles.items():
    download(filename, checksum=checksum, url_suffix="lab9-sql/")
    
print("(Auxiliary files appear to be ready.)")

Loading BokehJS ...

[https://cse6040.gatech.edu/datasets/lab9-sql/df_complaints_by_city_soln.csv]
==> 'resource/asnlib/publicdata/df_complaints_by_city_soln.csv' is already available.
==> Checksum test passes: b07d65c208bd791ea21679a3551ae265
==> 'resource/asnlib/publicdata/df_complaints_by_city_soln.csv' is ready!

[https://cse6040.gatech.edu/datasets/lab9-sql/df_complaints_by_hour_soln.csv]
==> 'resource/asnlib/publicdata/df_complaints_by_hour_soln.csv' is already available.
==> Checksum test passes: f06fcd917876d51ad52ddc13b2fee69e
==> 'resource/asnlib/publicdata/df_complaints_by_hour_soln.csv' is ready!

[https://cse6040.gatech.edu/datasets/lab9-sql/df_noisy_by_hour_soln.csv]
==> 'resource/asnlib/publicdata/df_noisy_by_hour_soln.csv' is already available.
==> Checksum test passes: 30f3fa7c753d4d3f4b3edfa1f6d05bcc
==> 'resource/asnlib/publicdata/df_noisy_by_hour_soln.csv' is ready!

[https://cse6040.gatech.edu/datasets/lab9-sql/df_plot_stacked_fraction_soln.csv]
==> 'resource/asnlib/publicdata/df_plot_

In [3]:
from nb9utils import make_barchart, make_stacked_barchart
from bokeh.io import show

In [4]:
def demo_bar():
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource
    data = [
        ['201720', 'cat1', 20],
        ['201720', 'cat2', 30],
        ['201720', 'cat3', 40],
        ['201721', 'cat1', 20],
        ['201721', 'cat2', 0],
        ['201721', 'cat3', 40],
        ['201722', 'cat1', 50],
        ['201722', 'cat2', 60],
        ['201722', 'cat3', 10],
    ]
    df = pd.DataFrame(data, columns=['week', 'category', 'count'])
    pt = df.pivot('week', 'category', 'count')
    pt.cumsum(axis=1)
    return df, pt

df_demo, pt_demo = demo_bar()
pt_demo

category,cat1,cat2,cat3
week,,,
201720,20,30,40
201721,20,0,40
201722,50,60,10


In [5]:
def demo_stacked_bar(pt):
    from bokeh.models.ranges import FactorRange
    from bokeh.io import show
    from bokeh.plotting import figure
    p = figure(title="count",
               x_axis_label='week', y_axis_label='category',
               x_range = FactorRange(factors=list(pt.index)),
               plot_height=500, plot_width=800)
    p.vbar(x=pt.index, bottom=0, top=pt.cat1, width=0.2, color='red', legend_label='cat1')
    p.vbar(x=pt.index, bottom=pt.cat1, top=pt.cat2+pt.cat1, width=0.2, color='blue', legend_label='cat2')
    p.vbar(x=pt.index, bottom=pt.cat2+pt.cat1,top=pt.cat3+pt.cat2+pt.cat1, width=0.2, color='green', legend_label='cat3')
    return p
    
show(demo_stacked_bar(pt_demo))

In [6]:
# Build a Pandas data frame
names = ['Bob','Jessica','Mary','John','Mel']
births = [968, 155, 77, 578, 973]
name_birth_pairs = list(zip(names, births))
baby_names = pd.DataFrame(data=name_birth_pairs, columns=['Names', 'Births'])
display(baby_names)

,Names,Births
0,Bob,968
1,Jessica,155
2,Mary,77
3,John,578
4,Mel,973


In [7]:
p = make_barchart(baby_names, 'Names', 'Births', kwargs_figure={'plot_width': 640, 'plot_height': 320})
show(p)

In [8]:
from nb9utils import download_nyc311db
DB_FILENAME = download_nyc311db()

[https://onedrive.live.com/download?cid=FD520DDC6BE92730&resid=FD520DDC6BE92730%21616&authkey=AEeP_4E1uh-vyDENYC-311-2M.db]
==> 'resource/asnlib/publicdata/NYC-311-2M.db' is already available.
==> Checksum test passes: f48eba2fb06e8ece7479461ea8c6dee9
==> 'resource/asnlib/publicdata/NYC-311-2M.db' is ready!



In [9]:
# Connect
import sqlite3 as db
disk_engine = db.connect('file:{}?mode=ro'.format(DB_FILENAME), uri=True)

In [10]:
import time

print ("Reading ...")
start_time = time.time ()

# Perform SQL query through the disk_engine connection.
# The return value is a pandas data frame.
df = pd.read_sql_query ('select * from data', disk_engine)

elapsed_time = time.time () - start_time
print ("==> Took %g seconds." % elapsed_time)

# Dump the first few rows
df.head()

Reading ...
==> Took 5.55707 seconds.


,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2015-09-15 02:14:04.000000,None,NYPD,Illegal Parking,Blocked Hydrant,None
1,2,2015-09-15 02:12:49.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
2,3,2015-09-15 02:11:19.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
3,4,2015-09-15 02:09:46.000000,None,NYPD,Noise - Commercial,Loud Talking,BRONX
4,5,2015-09-15 02:08:01.000000,2015-09-15 02:08:18.000000,DHS,Homeless Person Assistance,Status Call,NEW YORK


In [11]:
query = '''
  SELECT *
    FROM data
    LIMIT 5
'''
start_time = time.time ()
df = pd.read_sql_query (query, disk_engine)
elapsed_time = time.time () - start_time
print ("==> LIMIT version took %g seconds." % elapsed_time)

df

==> LIMIT version took 0.0382552 seconds.


,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2015-09-15 02:14:04.000000,None,NYPD,Illegal Parking,Blocked Hydrant,None
1,2,2015-09-15 02:12:49.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
2,3,2015-09-15 02:11:19.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
3,4,2015-09-15 02:09:46.000000,None,NYPD,Noise - Commercial,Loud Talking,BRONX
4,5,2015-09-15 02:08:01.000000,2015-09-15 02:08:18.000000,DHS,Homeless Person Assistance,Status Call,NEW YORK


In [12]:
query = 'SELECT DISTINCT City FROM data'
df = pd.read_sql_query(query, disk_engine)

print("Found {} unique cities. The first few are:".format(len(df)))
df.head()

Found 547 unique cities. The first few are:


,City
0,None
1,NEW YORK
2,BRONX
3,STATEN ISLAND
4,ELMHURST


In [13]:
query = '''
  SELECT ComplaintType, Descriptor, Agency
    FROM data
    GROUP BY ComplaintType
'''

df = pd.read_sql_query(query, disk_engine)
print(df.shape)
df.head()


(200, 3)


,ComplaintType,Descriptor,Agency
0,AGENCY,HOUSING QUALITY STANDARDS,HPD
1,APPLIANCE,ELECTRIC/GAS RANGE,HPD
2,Adopt-A-Basket,10A Adopt-A-Basket,DSNY
3,Agency Issues,Bike Share,DOT
4,Air Quality,"Air: Odor/Fumes, Vehicle Idling (AD3)",DEP


In [14]:
query = '''
  SELECT ComplaintType, COUNT(*)
    FROM data
    GROUP BY ComplaintType
    LIMIT 10
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,ComplaintType,COUNT(*)
0,AGENCY,2
1,APPLIANCE,11263
2,Adopt-A-Basket,50
3,Agency Issues,7428
4,Air Quality,8151


In [15]:
query = '''
  SELECT LOWER(ComplaintType), LOWER(Descriptor), LOWER(Agency)
    FROM data
    GROUP BY LOWER(ComplaintType)
    LIMIT 10
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,LOWER(ComplaintType),LOWER(Descriptor),LOWER(Agency)
0,adopt-a-basket,10a adopt-a-basket,dsny
1,agency,housing quality standards,hpd
2,agency issues,bike share,dot
3,air quality,"air: odor/fumes, vehicle idling (ad3)",dep
4,animal abuse,other (complaint details),nypd


In [16]:
query0 = "SELECT DISTINCT ComplaintType FROM data"
df0 = pd.read_sql_query(query0, disk_engine)
print("Found {} unique `ComplaintType` strings.".format(len(df0)))
display(df0.head())

query1 = "SELECT DISTINCT LOWER(ComplaintType) FROM data"
df1 = pd.read_sql_query(query1, disk_engine)
print("\nFound {} unique `LOWER(ComplaintType)` strings.".format(len(df1)))
display(df1.head())

print("\n==> Therefore, there are {} cases that are duplicated. Which ones?".format(len(df0) - len(df1)))

Found 200 unique `ComplaintType` strings.


,ComplaintType
0,Illegal Parking
1,Noise - Street/Sidewalk
2,Noise - Commercial
3,Homeless Person Assistance
4,Highway Condition



Found 198 unique `LOWER(ComplaintType)` strings.


,LOWER(ComplaintType)
0,illegal parking
1,noise - street/sidewalk
2,noise - commercial
3,homeless person assistance
4,highway condition



==> Therefore, there are 2 cases that are duplicated. Which ones?


In [17]:
query2 = '''
    SELECT ComplaintType, COUNT(*)
      FROM (SELECT DISTINCT ComplaintType FROM data)
      GROUP BY LOWER(ComplaintType)
      HAVING COUNT(*) >= 2
'''
df2 = pd.read_sql_query(query2, disk_engine)
df2

,ComplaintType,COUNT(*)
0,Elevator,2
1,PLUMBING,2


In [18]:
query = '''
    SELECT DISTINCT ComplaintType
      FROM data
      WHERE LOWER(ComplaintType) IN ("plumbing", "elevator")
'''
df = pd.read_sql_query(query, disk_engine)
df.head()

,ComplaintType
0,PLUMBING
1,Elevator
2,Plumbing
3,ELEVATOR


In [19]:
query = '''
  SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY Agency
'''
df = pd.read_sql_query(query, disk_engine)
df.head()

,Agency,NumComplaints
0,3-1-1,1289
1,ACS,3
2,AJC,6
3,CAU,1
4,CCRB,1


In [20]:
query = '''
  SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY UPPER(Agency)
    ORDER BY NumComplaints
'''
df = pd.read_sql_query(query, disk_engine)
df.tail()

,Agency,NumComplaints
45,DSNY,152004
46,DEP,181121
47,DOT,322969
48,NYPD,340694
49,HPD,640096


In [21]:
query = '''
  SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY UPPER(Agency)
    ORDER BY -NumComplaints
'''

# Alternative: query =
'''
SELECT Agency, COUNT(*) AS NumComplaints 
    FROM data 
    GROUP BY UPPER(Agency)
    ORDER BY NumComplaints DESC 
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,Agency,NumComplaints
0,HPD,640096
1,NYPD,340694
2,DOT,322969
3,DEP,181121
4,DSNY,152004


In [21]:
query = '''
  SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY UPPER(Agency)
    ORDER BY -NumComplaints
'''

# Alternative: query =
'''
SELECT Agency, COUNT(*) AS NumComplaints 
    FROM data 
    GROUP BY UPPER(Agency)
    ORDER BY NumComplaints DESC 
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,Agency,NumComplaints
0,HPD,640096
1,NYPD,340694
2,DOT,322969
3,DEP,181121
4,DSNY,152004


In [22]:
p = make_barchart(df[:20], 'Agency', 'NumComplaints',
                  {'title': 'Top 20 agencies by number of complaints',
                   'plot_width': 800, 'plot_height': 320})
p.xaxis.major_label_orientation = 0.66
show(p)

In [33]:
del query # clears any existing `query` variable; you should define it, below!

# Define a variable named `query` containing your solution
query = '''
  SELECT LOWER(ComplaintType) AS type, COUNT(ComplaintType) AS freq
    FROM data
    GROUP BY type
    ORDER BY -freq
'''

# Runs your `query`:
df_complaint_freq = pd.read_sql_query(query, disk_engine)
df_complaint_freq.head()

,type,freq
0,heat/hot water,241430
1,street condition,124347
2,street light condition,98577
3,blocked driveway,95080
4,illegal parking,83961


In [35]:
p = make_barchart(df_complaint_freq[:25], 'type', 'freq',
                  {'title': 'Top 25 complaints by type',
                   'plot_width': 800, 'plot_height': 320})
p.xaxis.major_label_orientation = 0.66
show(p)

In [36]:
query = '''
  SELECT LOWER(ComplaintType) AS type, COUNT(*) AS freq
    FROM data
    WHERE LOWER(ComplaintType) LIKE '%noise%'
    GROUP BY type
    ORDER BY -freq
'''

df_noisy = pd.read_sql_query(query, disk_engine)
print("Found {} queries with 'noise' in them.".format(len(df_noisy)))
df_noisy

Found 8 queries with 'noise' in them.


,type,freq
0,noise,54165
1,noise - street/sidewalk,48436
2,noise - commercial,42422
3,noise - vehicle,18370
4,noise - park,4020
5,noise - helicopter,1715
6,noise - house of worship,1143
7,collection truck noise,184


In [40]:
del query # define a new `query` variable, below

query = '''
  SELECT UPPER(City) AS name, COUNT(*) AS freq
    FROM data
    GROUP BY name
    ORDER BY -freq LIMIT 10
'''

# Runs your `query`:
df_whiny_cities = pd.read_sql_query(query, disk_engine)
df_whiny_cities

,name,freq
0,BROOKLYN,579363
1,NEW YORK,385655
2,BRONX,342533
3,None,168692
4,STATEN ISLAND,92509
5,JAMAICA,46683
6,FLUSHING,35504
7,ASTORIA,31873
8,RIDGEWOOD,21618
9,WOODSIDE,15932


In [42]:
query = '''
  SELECT UPPER(City) AS name, COUNT(*) AS freq
    FROM data
    WHERE name <> 'None'
    GROUP BY City COLLATE NOCASE
    ORDER BY -freq
    LIMIT 10
'''
df_whiny_cities2 = pd.read_sql_query(query, disk_engine)
df_whiny_cities2

,name,freq
0,BROOKLYN,579363
1,NEW YORK,385655
2,BRONX,342533
3,STATEN ISLAND,92509
4,JAMAICA,46683
5,FLUSHING,35504
6,ASTORIA,31873
7,RIDGEWOOD,21618
8,WOODSIDE,15932
9,CORONA,15740


In [43]:
TOP_CITIES = list(df_whiny_cities2.head(7)['name'])
TOP_CITIES

['BROOKLYN',
 'NEW YORK',
 'BRONX',
 'STATEN ISLAND',
 'JAMAICA',
 'FLUSHING',
 'ASTORIA']

In [49]:
def strs_to_args(str_list):
    assert type (str_list) is list
    assert all ([type (s) is str for s in str_list])
    final_str = ['"' + s + '"' for s in str_list]
    return ', '.join(final_str)


In [60]:
query = '''
  SELECT UPPER(City) AS city_name, COUNT(*) AS complaint_count, LOWER(ComplaintType) as complaint_type
    FROM data
    WHERE city_name in ("BROOKLYN", "NEW YORK", "BRONX", "STATEN ISLAND", "JAMAICA", "FLUSHING", "ASTORIA")
    GROUP BY city_name, complaint_type
    ORDER BY city_name, complaint_count
'''

df_complaints_by_city = pd.read_sql_query(query, disk_engine)

# Previews the results of your query:
print("Found {} records.".format(len(df_complaints_by_city)))
display(df_complaints_by_city.head(10))

Found 1042 records.


,city_name,complaint_count,complaint_type
0,ASTORIA,1,bottled water
1,ASTORIA,1,bridge condition
2,ASTORIA,1,city vehicle placard complaint
3,ASTORIA,1,open flame permit
4,ASTORIA,1,panhandling
5,ASTORIA,1,stalled sites
6,ASTORIA,1,window guard
7,ASTORIA,2,beach/pool/sauna complaint
8,ASTORIA,2,cranes and derricks
9,ASTORIA,2,drinking water


In [62]:
# Let's consider only the top 25 complaints (by total)
top_complaints = df_complaint_freq[:25]
print("Top complaints:")
display(top_complaints)

Top complaints:


,type,freq
0,heat/hot water,241430
1,street condition,124347
2,street light condition,98577
3,blocked driveway,95080
4,illegal parking,83961
5,unsanitary condition,81394
6,paint/plaster,69929
7,water system,69209
8,plumbing,60105
9,noise,54165


In [122]:
# Plot subset of data corresponding to the top complaints
df_plot = top_complaints.merge(df_complaints_by_city,
                               left_on=['type'],
                               right_on=['complaint_type'],
                               how='left')
df_plot.dropna(inplace=True)
print("Data to plot (first few rows):")
display(df_plot.head())
print("...")

Data to plot (first few rows):


,type,freq,city_name,complaint_count,complaint_type
0,heat/hot water,241430,ASTORIA,3396.0,heat/hot water
1,heat/hot water,241430,BRONX,79690.0,heat/hot water
2,heat/hot water,241430,BROOKLYN,72410.0,heat/hot water
3,heat/hot water,241430,FLUSHING,2741.0,heat/hot water
4,heat/hot water,241430,JAMAICA,3376.0,heat/hot water


...


In [123]:
# Some code to render a Bokeh stacked bar chart

kwargs_figure = {'title': "Distribution of the top 25 complaints among top 7 cities with the most complaints",
                 'width': 800,
                 'height': 400,
                 'tools': "hover,crosshair,pan,box_zoom,wheel_zoom,save,reset,help"}

def plot_complaints_stacked_by_city(df, y='complaint_count'):
    p = make_stacked_barchart(df, 'complaint_type', 'city_name', y,
                              x_labels=list(top_complaints['type']), bar_labels=TOP_CITIES,
                              kwargs_figure=kwargs_figure)
    p.xaxis.major_label_orientation = 0.66
    from bokeh.models import HoverTool
    hover_tool = p.select(dict(type=HoverTool))
    hover_tool.tooltips = [("y", "$y{int}")]
    return p

show(plot_complaints_stacked_by_city(df_plot))

In [136]:
df_plot_fraction = df_plot.copy()

df_plot_fraction['complaint_frac'] = df_plot_fraction.apply(
    lambda row: row['complaint_count'] / row['freq'],
    axis=1
)
df_plot_fraction = df_plot_fraction.drop(columns=['complaint_count'])

df_plot_fraction.head()


,type,freq,city_name,complaint_type,complaint_frac
0,heat/hot water,241430,ASTORIA,heat/hot water,0.014066
1,heat/hot water,241430,BRONX,heat/hot water,0.330075
2,heat/hot water,241430,BROOKLYN,heat/hot water,0.299921
3,heat/hot water,241430,FLUSHING,heat/hot water,0.011353
4,heat/hot water,241430,JAMAICA,heat/hot water,0.013983


In [138]:
query = '''
  SELECT LOWER(ComplaintType), CreatedDate, UPPER(City)
    from data
    where CreatedDate >= "2015-09-15 00:00:00.0"
      and CreatedDate < "2015-09-16 00:00:00.0"
    order by CreatedDate
'''
df = pd.read_sql_query (query, disk_engine)
df

,LOWER(ComplaintType),CreatedDate,UPPER(City)
0,illegal parking,2015-09-15 00:01:23.000000,None
1,blocked driveway,2015-09-15 00:02:29.000000,REGO PARK
2,taxi complaint,2015-09-15 00:02:34.000000,NEW YORK
3,opinion for the mayor,2015-09-15 00:03:07.000000,None
4,opinion for the mayor,2015-09-15 00:03:07.000000,None
...,...,...,...
113,homeless person assistance,2015-09-15 02:08:01.000000,NEW YORK
114,noise - commercial,2015-09-15 02:09:46.000000,BRONX
115,noise - street/sidewalk,2015-09-15 02:11:19.000000,NEW YORK
116,noise - street/sidewalk,2015-09-15 02:12:49.000000,NEW YORK


In [139]:
query = '''
  SELECT CreatedDate, STRFTIME('%H', CreatedDate) AS Hour, LOWER(ComplaintType)
    FROM data
    LIMIT 5
'''
df = pd.read_sql_query (query, disk_engine)
df

,CreatedDate,Hour,LOWER(ComplaintType)
0,2015-09-15 02:14:04.000000,02,illegal parking
1,2015-09-15 02:12:49.000000,02,noise - street/sidewalk
2,2015-09-15 02:11:19.000000,02,noise - street/sidewalk
3,2015-09-15 02:09:46.000000,02,noise - commercial
4,2015-09-15 02:08:01.000000,02,homeless person assistance


In [140]:
# Your task: Construct `df_complaints_by_hour` as directed.
query = '''
    select strftime ('%H', CreatedDate) as hour, count(*) as count
    from data group by hour
'''

df_complaints_by_hour = pd.read_sql_query(query, disk_engine)
# Displays your answer:
display(df_complaints_by_hour)

,hour,count
0,00,564703
1,01,23489
2,02,15226
3,03,10164
4,04,8692
5,05,10224
6,06,23051
7,07,42273
8,08,73811
9,09,100077


In [142]:
p = make_barchart(df_complaints_by_hour, 'hour', 'count',
                  {'title': 'Complaints by hour',
                   'plot_width': 800, 'plot_height': 320})
show(p)

In [143]:
query = '''
  SELECT COUNT(*)
    FROM data
    WHERE STRFTIME('%H:%M:%f', CreatedDate) = '00:00:00.000'
'''

pd.read_sql_query(query, disk_engine)

,COUNT(*)
0,532285


In [154]:
query = '''
    select strftime ('%H', CreatedDate) as hour, count(*) as count
    from data
    WHERE STRFTIME('%H:%M:%f', CreatedDate) <> '00:00:00.000' and LOWER(ComplaintType) LIKE '%noise%'
    group by hour
'''

df_noisy_by_hour = pd.read_sql_query(query, disk_engine)
display(df_noisy_by_hour)

,hour,count
0,00,15349
1,01,11284
2,02,7170
3,03,4241
4,04,3083
5,05,2084
6,06,2832
7,07,3708
8,08,4553
9,09,5122


In [156]:
p = make_barchart(df_noisy_by_hour, 'hour', 'count',
                  {'title': 'Noise complaints by hour',
                   'plot_width': 800, 'plot_height': 320})
show(p)